In [1]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from os.path import join

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/tensorflow/tpu/

Cloning into 'tpu'...
remote: Enumerating objects: 10883, done.
remote: Counting objects: 100% (1085/1085), done.
remote: Compressing objects: 100% (526/526), done.
remote: Total 10883 (delta 591), reused 969 (delta 535), pack-reused 9798
Receiving objects: 100% (10883/10883), 45.76 MiB | 28.98 MiB/s, done.
Resolving deltas: 100% (7629/7629), done.


In [4]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
import fnmatch,re
import sys
%tensorflow_version 1.x
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import segmentation

TensorFlow 1.x selected.
TPU address is grpc://10.39.150.50:8470

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3459755694497704833),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12798286896174456250),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16976239080083396096),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12957314454462010221),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9562426773732518700),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 10802543230594935609),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9207295920382530566),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11906828515381211153),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU

In [5]:
doSegmentation = True

In [6]:
import json
json_dir= 'drive/MyDrive/mcfd_json_files/mcfd.json'
delay_dir ='drive/MyDrive/mcfd_json_files/delay_mcfd.json'

In [7]:
with open(json_dir,'r') as json_file:
  mcfd = json.load(json_file)
with open(delay_dir , 'r') as delay_file:
  delay = json.load(delay_file)

In [ ]:
print(mcfd)

{'1': {'segment': [{'start': 874, 'end': 1011, 'code': 1}, {'start': 1012, 'end': 1079, 'code': 6}, {'start': 1080, 'end': 1108, 'code': 2}, {'start': 1109, 'end': 1285, 'code': 3}], 'camera_reference': 11}, '2': {'segment': [{'start': 308, 'end': 374, 'code': 1}, {'start': 375, 'end': 399, 'code': 2}, {'start': 400, 'end': 600, 'code': 3}], 'camera_reference': 4}, '3': {'segment': [{'start': 380, 'end': 590, 'code': 1}, {'start': 591, 'end': 625, 'code': 2}, {'start': 626, 'end': 784, 'code': 3}], 'camera_reference': 11}, '4': {'segment': [{'start': 230, 'end': 287, 'code': 1}, {'start': 288, 'end': 314, 'code': 2}, {'start': 315, 'end': 380, 'code': 3}, {'start': 381, 'end': 600, 'code': 6}, {'start': 601, 'end': 638, 'code': 2}, {'start': 639, 'end': 780, 'code': 3}], 'camera_reference': 6}, '5': {'segment': [{'start': 288, 'end': 310, 'code': 1}, {'start': 311, 'end': 336, 'code': 2}, {'start': 337, 'end': 450, 'code': 3}], 'camera_reference': 11}, '6': {'segment': [{'start': 325, 

In [ ]:
 #it's for understanding 
temp = []
for x in range(1, 25):
  s = mcfd[str(x)]['segment']
  for i in s:
    if i["code"]==2:
      temp.append(i['end'] - i['start'])

In [ ]:
temp

[28,
 24,
 34,
 26,
 37,
 25,
 46,
 31,
 27,
 23,
 18,
 25,
 48,
 40,
 34,
 32,
 49,
 40,
 30,
 101,
 127,
 37,
 41,
 75,
 40]

In [ ]:
#this code is for understanding only
dir = 'drive/MyDrive/Multiple cameras fall dataset/'
save_dir = 'drive/MyDrive/mcfd_processed_data/'
folders = os.listdir(dir)
folders.sort()
i=1
fall= 1
non_fall= 1
for folder in folders:
  folder_path = os.path.join(dir, folder)
  videos = os.listdir(folder_path)
  videos.sort()
  segments = mcfd[str(i)]['segment']
  i=i+1
  for video in videos: 
    video = os.path.join(folder_path, video)
    print(video)
    for item in segments:
      start = item['start']
      end = item['end']
      count=0
      cap = cv2.VideoCapture(video)
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      frames = []
      while (count<total_frames):
          count = count+1
          ret, frame = cap.read()
          frame_pos= cap.get(cv2.CAP_PROP_POS_FRAMES)
          if frame_pos in range(start, end):
            frames.append(frame)
      size = frame.shape[0],frame.shape[1]
      #fall.append(fall_frames)
      path = ''
      if item['code']==2:
        path  = os.path.join(save_dir, 'fall/falling'+ str(fall)+ '.avi')
        fall = fall+1
      else:-
        path = os.path.join(save_dir, 'non_fall/not_falling'+ str(non_fall)+'.avi')
        non_fall = non_fall+1
      out = cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*'DIVX'), 120, size)
      for f in range(len(frames)):
          out.write(frames[f])
      out.release()



KeyboardInterrupt: ignored

In [ ]:
already_processed = 0
dir = 'drive/MyDrive/Multiple cameras fall dataset/'
if doSegmentation:
  save_dir = 'drive/MyDrive/mcfd_segmented data/'
else:
  save_dir = 'drive/MyDrive/mcfd_processed_data/'

if not os.path.isdir(save_dir+'fall'):
  os.mkdir(save_dir+'fall')
if not os.path.isdir(save_dir+'non_fall'):
  os.mkdir(save_dir+'non_fall')
folders = os.listdir(dir)
folders.sort()
fall= 1
non_fall= 1
fps = 10
cntr = 0
for scenario_number, folder in enumerate(folders):
  scenario_number = scenario_number+1
  folder_path = os.path.join(dir, folder)
  videos = os.listdir(folder_path)
  videos.sort()
  segments = mcfd[str(scenario_number)]['segment']
  for camera_num, video in enumerate(videos):
    if cntr<already_processed:
      cntr +=1
      continue 
    camera_num= camera_num+1
    d = delay['camera'+str(camera_num)][str(scenario_number)]
    video = os.path.join(folder_path, video)
    cap = cv2.VideoCapture(video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    segment_count = 0
    count = 0
    item = segments[segment_count]
    start = item['start'] +d
    end = item['end']+ d
    while (count<total_frames):
      count = count+1
      ret, frame = cap.read()
      if doSegmentation==True and count in range(start, end):
        frame = segmentation.extract_person(frame, extract_all=True)
      if count>=end and len(frames)==0:
        print("ERROR: on {} segment {}".format(video,segment_count))
        segment_count +=1
        if segment_count>=len(segments):
          break
        item = segments[segment_count]
        start = item['start']+d
        end = item['end']+d
      if count>=end and len(frames)>0:
        size = frame.shape[1],frame.shape[0]
        path = ''
        if item['code']==2:
          path  = os.path.join(save_dir, 'fall/mcfd_fall'+ str(fall)+ '.avi')
          fall = fall+1
        else:
          path = os.path.join(save_dir, 'non_fall/mcfd_non_fall'+ str(non_fall)+'.avi')
          non_fall = non_fall+1
        
        out = cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
        for f in range(len(frames)):
            out.write(frames[f])
        out.release()
        segment_count +=1
        if segment_count>=len(segments):
          break
        item = segments[segment_count]
        start = item['start']+d
        end = item['end']+d
        frames = []
      if count in range(start, end):
        frames.append(frame)
    cntr = cntr + 1
    
    print("extracted from: "+video)
    print("video number: "+str(cntr))

extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam1.avi
video number: 1
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam2.avi
video number: 2
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam3.avi
video number: 3
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam4.avi
video number: 4
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam5.avi
video number: 5
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam6.avi
video number: 6
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam7.avi
video number: 7
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute01/cam8.avi
video number: 8
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute02/cam1.avi
video number: 9
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute02/cam2.avi
video number: 10
extracted from: drive/MyDrive/Multiple cameras fall dataset/chute02/c

Total position  = 163\
Fall = 25\
Non Fall = 138\
total generated video = 163 * 8 = 1344\
Fall Video = 25 * 8 = 200\
Non Fall Video = 138 * 8 =  1104